In [108]:
#coding:urf-8
from WindPy import w
import pandas as pd
import numpy as np
from datetime import datetime
w.start()

#返回板块成分股，默认返回全部A股代码，返回格式为list
def getBKChengFen(td,sectorid= "a001010100000000"):
    wsd_data = w.wset("sectorconstituent","date="+td+";sectorid=" +sectorid)
    df = pd.DataFrame(wsd_data.Data,wsd_data.Fields).T
    return list(df.wind_code)

#返回实时行情数据，注意每次只能返回一个指标，多了取不到数据
def getWsqData(tickers,fields):
    wsq_data = w.wsq(tickers,fields)
    df = pd.DataFrame(np.array(wsq_data.Data).T,index = wsq_data.Codes,columns=wsq_data.Fields)
    return df

#返回多个指标的实时行情DataFrame
def getMulWSQData(tickers,fields):
    dfs = [getWsqData(tickers,field) for field in fields]
    return pd.concat(dfs,axis=1)

#返回分钟行情数据
def getWSIData(tickers,fields,s_td,e_td,barSize="30"):
    wsi_data = w.wsi(tickers,fields,s_td,e_td,"BarSize="+str(barSize))
    df = pd.DataFrame(np.array(wsi_data).T,columns=wsi_data.Fields)
    return df

#返回多维数据
def getWSSData(tickers,fields,td):
    wss_data = w.wss(tickers,fields,"tradeDate="+td)
    df = pd.DataFrame(np.array(wss_data.Data).T,index = wss_data.Codes,columns=wss_data.Fields)
    return df

#返回前一天同时刻的全市场成交额
def getPreAmt(df_preAmt,td_time):
    preID = 0
    if td_time<'10:00':
        preID="NA"
    elif td_time<"10:30":
        preID=0
    elif td_time<"11:00":
        preID=1
    elif td_time<"11:30":
        preID=2
    elif td_time<"13:30":
        preID=3
    elif td_time<"14:00":
        preID=4
    elif td_time<"14:30":
        preID=5
    elif td_time<"15:00":
        preID=6
    else:
        preID=7    
    return df_preAmt.at[preID,"amount"]


#返回成交金额分数
def getAmtScore(td_amt,pre_amt):
    amtScore = 0
    pct = td_amt/pre_amt-1
    if pct>=0.5:
        amtScore = 100
    elif pct>=-0.5:
        amtScore = pct/0.5*100
    else:
        amtScore=-100
    return amtScore

#计算中位数涨幅得分
def getMedianScore(pct_median):
    score = 0
    if pct_median>0.03:
        score = 100
    elif pct_median>0:
        score = pct_median/0.03*100
    else:
        score = 0
    return score

#计算连扳和破板合计得分
def getLmtConBreakScore(cn,bk):
    score = 0
    if cn+bk>100:
        score=100
    elif cn+bk>0:
        score = (cn+bk)*0.025
    else:
        score=0
    return score

#返回市场状态，上涨个数、下跌个数、涨幅大于7%个数、跌幅大于7%个数，涨停个数、跌停个数、涨幅中位数等
def getStatus(df,st_sec,td,pre_amt):
    item = {}
    #上涨个数、下跌个数、涨幅大于7%个数、跌幅大于7%个数，涨停个数、跌停个数
    pcg_up = df[df["RT_PCT_CHG"]>0]["RT_PCT_CHG"].count()
    pcg_dn = df[df["RT_PCT_CHG"]<0]["RT_PCT_CHG"].count()
    pcg_up_n = df[df["RT_PCT_CHG"]>0.07]["RT_PCT_CHG"].count()
    pcg_dn_n = df[df["RT_PCT_CHG"]<-0.07]["RT_PCT_CHG"].count()
    pcg_up_lmt = df[(df["RT_PCT_CHG"]>0.099) | (df[df.index.isin(st_sec)]["RT_PCT_CHG"]>0.049)]
    pcg_dn_lmt = df[(df["RT_PCT_CHG"]<-0.099) | (df[df.index.isin(st_sec)]["RT_PCT_CHG"]<-0.049)]
    pcg_up_lmt_n = pcg_up_lmt["RT_PCT_CHG"].count()
    pcg_dn_lmt_n = pcg_dn_lmt["RT_PCT_CHG"].count()
    
    #连板数
    pre_30d = w.tdaysoffset(-30, td, "").Data[0][0]
    cont_lmt = pcg_up_lmt[(pcg_up_lmt["MAXUPORDOWN"]==1)&(pcg_up_lmt["IPO_DATE"]< pre_30d)]["RT_PCT_CHG"].count()
    
    #涨停板打开
    df["high_pct_chg"] = df["RT_HIGH"]/df["RT_PRE_CLOSE"]-1
    td_up_lmt = df[(df["high_pct_chg"]>0.099) | (df[df.index.isin(st_sec)]["high_pct_chg"]>0.049)]
    break_up_lmt = td_up_lmt[~td_up_lmt.index.isin(pcg_up_lmt.index)]["RT_PCT_CHG"].count()
    
    #涨幅中位数
    pcg_median = df[df["RT_HIGH"]>0]["RT_PCT_CHG"].median()
    
    #成交金额
    td_amt = df["RT_AMT"].sum()
    amtScore = getAmtScore(td_amt,pre_amt)
    
    item["pcg_up"] = pcg_up                     
    item['pcg_dn'] = pcg_dn
    item['pcg_up_n'] = pcg_up_n
    item['pcg_dn_n'] = pcg_dn_n
    item['pcg_up_lmt'] = pcg_up_lmt_n
    item['pcg_dn_lmt'] = pcg_dn_lmt_n
    item['pcg_median'] = pcg_median
    item['cont_lmt'] = cont_lmt
    item['break_up_lmt'] = break_up_lmt
    
    item['pcg_up_n_score'] = 100 if pcg_up_n>75 else pcg_up_n/75.0
    item['pcg_dn_n_score'] = -100 if pcg_dn_n>75 else pcg_dn_n/75.0
    item['pcg_up_lmt_score'] = 100 if pcg_up_lmt_n>50 else pcg_up_lmt_n/50.0
    item['pcg_dn_lmt_score'] = -100 if pcg_dn_lmt_n>300 else pcg_dn_lmt_n/50.0*(-1)
    item['pcg_median_score'] = getMedianScore(pcg_median)
    item['pcg_chg_score'] = pcg_up/float((pcg_up+pcg_dn)) if pcg_up>pcg_dn else 2* pcg_up/float((pcg_up+pcg_dn))*100
    item['amtScore'] = amtScore

    #计算总得分
    s1 = (item['pcg_up_lmt_score']+item['pcg_dn_lmt_score'])*0.05 if (item['pcg_up_lmt_score']+item['pcg_dn_lmt_score'])>0 else 0
    s2 = (item['pcg_up_n_score']+item['pcg_dn_n_score'])*0.15 if (item['pcg_up_n_score']+item['pcg_dn_n_score'])>0 else 0
    s3 =  item['pcg_median_score']*0.15
    s4 = item['pcg_chg_score']*0.6
    s5 = amtScore*0.025
    s6 = getLmtConBreakScore(cont_lmt,break_up_lmt)
    item['fullScore'] = s1+s2+s3+s4+s5+s6
    
    return item
    
    pass


#sectorid = "1000000087000000"       #sz50
sectorid = '1000006526000000'       #st股票
#sectorid = "a001010100000000"

now = datetime.now()
td = now.strftime('%Y%m%d')                         #date  'YYYYMMDD'
pre_td = w.tdaysoffset(-1, td, "").Data[0][0].strftime('%Y%m%d')      #前一个交易日
td_time = now.strftime('%H%M')                    #time   'hh:mm'

tickers = getBKChengFen(td)                     #当天的全部A股代码
st_sec = getBKChengFen(td,sectorid)            #当天的全部st股票代码

field_1 =  "rt_date,rt_time,rt_pre_close,rt_high,rt_last,rt_amt,rt_pct_chg".split(",")    #需要获取的实时行情的字段
field_2 = ["ipo_date","maxupordown"]
df_wsq = getMulWSQData(tickers,field_1)               #返回实时行情的日期、时间、前收盘价、最高价、最新价、成交额、涨跌幅
df_wss = getWSSData(tickers,field_2,pre_td)           #返回上市日、前一天的涨跌停状态
df = pd.concat([df_wsq,df_wss],axis=1)                #合并两个DataFrame

df_preAmt = pd.read_excel("AllStockAmtSum.xlsx")
pre_amt = getPreAmt(df_preAmt,td_time)
items = getStatus(df,st_sec,td,pre_amt)

for k,v in items.iteritems():
    print k+":"+str(v)




pcg_median:0.0098
cont_lmt:2
pcg_dn:368
pcg_up_lmt_score:100
pcg_median_score:32.6666666667
pcg_dn_lmt_score:-0.04
amtScore:100
pcg_dn_n:3
break_up_lmt:12
pcg_up_lmt:59
pcg_up_n_score:100
pcg_dn_n_score:0.04
pcg_dn_lmt:2
pcg_up:2670
pcg_up_n:78
fullScore:28.2813206057
pcg_chg_score:0.878867676103


In [33]:
#coding:urf-8
from WindPy import w
import pandas as pd
import numpy as np

w.start()

def getBKChengFen(td,sectorid="a001010100000000"):
    wsd_data = w.wset("sectorconstituent","date="+td+";sectorid=" +sectorid)
    df = pd.DataFrame(wsd_data.Data,wsd_data.Fields)
    return df.T

def TickerSplit(tickers,m=50):
    n = tickers.__len__()
    if n>0:
        return [tickers[i:i+m] for i in xrange(0,n,m)]
    return 

def getWSIData(tickers,fields,s_td,e_td,barSize="30"):
    if not isinstance(barSize,str):
        barSize = str(barSize)
    wsi_data = w.wsi(tickers,fields,s_td,e_td,"BarSize="+barSize)
    df = pd.DataFrame(np.array(wsi_data.Data).T,columns=wsi_data.Fields)
    return df


td = "20170822"
#tickers = ['000001.SZ','600000.SH','600016.SH','300146.SZ']
tickers = list(getBKChengFen(td).wind_code)
tickerList = TickerSplit(tickers)
fields=["amt"]
s_td = "2017-08-22 09:00:00"
e_td = "2017-08-22 15:30:00"
barSize=30
items = []
print '---------start------'
for ticker in tickerList:
    print '-------ticker'
    df = getWSIData(ticker,fields,s_td,e_td,barSize)
    items.append(df)

df_all = pd.concat(items)
df_all.to_excel("stockAmt30MIN.xlsx","30minData")
sm = df_all.groupby("time").sum()
sm.to_excel("AllStockAmtSum.xlsx","sumresult")
print "ok"
    

---------start------
-------ticker
-------ticker
-------ticker
-------ticker
-------ticker
-------ticker
-------ticker
-------ticker
-------ticker
-------ticker
-------ticker
-------ticker
-------ticker
-------ticker
-------ticker
-------ticker
-------ticker
-------ticker
-------ticker
-------ticker
-------ticker
-------ticker
-------ticker
-------ticker
-------ticker
-------ticker
-------ticker
-------ticker
-------ticker
-------ticker
-------ticker
-------ticker
-------ticker
-------ticker
-------ticker
-------ticker
-------ticker
-------ticker
-------ticker
-------ticker
-------ticker
-------ticker
-------ticker
-------ticker
-------ticker
-------ticker
-------ticker
-------ticker
-------ticker
-------ticker
-------ticker
-------ticker
-------ticker
-------ticker
-------ticker
-------ticker
-------ticker
-------ticker
-------ticker
-------ticker
-------ticker
-------ticker
-------ticker
-------ticker
-------ticker
-------ticker
-------ticker
---------over----doop
ok
